In [1]:
import pandas as pd

In [2]:
train_bal = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_bal.csv")
train_non_bal = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_same_size_as_bal.csv")

test = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [3]:
train_bal.shape, train_non_bal.shape, test.shape

((318045, 2), (318045, 2), (200000, 2))

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
def tokenize(row):
    star = row['stars']
    text = row['text']
    return f"__label__{str(int(star))} {' '.join(nltk.word_tokenize(text.lower()))}".split(" ")

In [6]:
tokens = train_bal.sample(5, random_state=42).apply(tokenize, axis=1).to_list()

In [7]:
tokens[1]

['__label__1',
 'huffman',
 "'s",
 'has',
 'a',
 'great',
 'atmosphere',
 'and',
 'a',
 'great',
 'staff',
 '.',
 'i',
 'was',
 'also',
 'really',
 'impressed',
 'with',
 'the',
 'wonderful',
 'milkshake',
 'bar',
 'right',
 'in',
 'the',
 'middle',
 'of',
 'the',
 'restaurant',
 '.',
 'they',
 'even',
 'have',
 'a',
 'game',
 'room',
 '.',
 'that',
 'gave',
 'me',
 'a',
 'blast',
 'from',
 'the',
 'past',
 '.',
 'but',
 '...',
 '...',
 '...',
 '..',
 'we',
 'probably',
 'should',
 'have',
 'paid',
 'attention',
 'to',
 'the',
 'number',
 'of',
 'people',
 'in',
 'the',
 'restaurant',
 'when',
 'we',
 'arrived',
 '.',
 'there',
 'was',
 'absolutely',
 'no',
 'one',
 'there',
 '.',
 'and',
 'after',
 'our',
 'meals',
 'were',
 'given',
 'to',
 'us',
 ',',
 'we',
 'understood',
 'why',
 '.',
 'the',
 'portions',
 'were',
 'great',
 '.',
 'you',
 'definitely',
 'would',
 "'ve",
 'left',
 'the',
 'place',
 'full',
 'but',
 'we',
 'did',
 "n't",
 'even',
 'finish',
 'our',
 'food',
 '.',
 '

In [8]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [12]:
import csv
def save_csv(output_file, tokens):
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        csv_writer.writerows(tokens)

In [9]:
tokens_bal = train_bal.parallel_apply(tokenize, axis=1)

In [16]:
import time

In [17]:
start = time.time()
save_csv("tokens_bal.csv", tokens_bal.to_list())
time.time() - start

tokens_bal = None

In [10]:
tokens_non_bal = train_non_bal.parallel_apply(tokenize, axis=1)

Process ForkPoolWorker-8:
Traceback (most recent call last):
Process ForkPoolWorker-10:
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandarallel/pandarallel.py", line 59, in global_worker
    return _func(x)
  File "/home/ec2-user/anaconda3/envs/py

KeyboardInterrupt: 

KeyboardInterrupt
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/nltk/tokenize/__init__.py", line 95, in sent_tokenize
    return tokenizer.tokenize(text)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/nltk/tokenize/punkt.py", line 1241, in tokenize
    return list(self.sentences_from_text(text, realign_boundaries))
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/nltk/tokenize/punkt.py", line 1291, in sentences_from_text
    return [text[s:e] for s, e in self.span_tokenize(text, realign_boundaries)]
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/nltk/tokenize/punkt.py", line 1291, in <listcomp>
    return [text[s:e] for s, e in self.span_tokenize(text, realign_boundaries)]
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/nltk/tokenize/punkt.py", line 1281, in span_tokenize
    for sl in slices:
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/si

In [ ]:
tokens_test = test.parallel_apply(tokenize, axis=1)

In [13]:
tokens_bal_output = "tokens_bal.csv"
tokens_non_bal_output = "tokens_non_bal.csv"
test_output = "test.csv"

In [14]:
save_csv("test.csv", tokens_bal.to_list()[:10])

In [ ]:
save_csv("test2.csv", tokens_bal[:10])